#MURAT ŞİMŞEK- MLOPS COUSE WEEK-4 DEPLOYMENT HW

In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2


In [2]:
import pickle
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
with open('/content/drive/MyDrive/mlops/model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [6]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [7]:
year = 2021
month = 2
output_file = f'fhv_tripdata_{year:04d}-{month:02d}_predicted.parquet'

In [8]:
df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year:04d}-{month:02d}.parquet')

In [9]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [10]:
y_pred.mean()

16.191691679979066

QUESTION-1:   16.19

In [12]:
df_result = pd.DataFrame()
df_result['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df_result['predicted_duration'] = y_pred

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [13]:
!ls -al


total 19272
drwxr-xr-x 1 root root     4096 Jun 27 12:59 .
drwxr-xr-x 1 root root     4096 Jun 27 12:55 ..
drwxr-xr-x 4 root root     4096 Jun 15 13:41 .config
drwx------ 5 root root     4096 Jun 27 12:56 drive
-rw-r--r-- 1 root root 19711507 Jun 27 12:59 fhv_tripdata_2021-02_predicted.parquet
drwxr-xr-x 1 root root     4096 Jun 15 13:42 sample_data


QUESTION 2: 19M

QUESTION 3: jupyter nbconvert --to script starter.ipynb